# **Fine-tuning XLS-R for Multi-Lingual ASR with 🤗 Transformers**

ref: https://huggingface.co/blog/fine-tune-xlsr-wav2vec2
Original colab link: https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb


**Wav2Vec2** is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by *Alexei Baevski, Michael Auli, and Alex Conneau*.  Soon after the superior performance of Wav2Vec2 was demonstrated on one of the most popular English datasets for ASR, called [LibriSpeech](https://huggingface.co/datasets/librispeech_asr), *Facebook AI* presented a multi-lingual version of Wav2Vec2, called [XLSR](https://arxiv.org/abs/2006.13979). XLSR stands for *cross-lingual speech representations* and refers to model's ability to learn speech representations that are useful across multiple languages.

XLSR's successor, simply called **XLS-R** (refering to the [*''XLM-R*](https://ai.facebook.com/blog/-xlm-r-state-of-the-art-cross-lingual-understanding-through-self-supervision/) *for Speech''*), was released in [November 2021](https://ai.facebook.com/blog/xls-r-self-supervised-speech-processing-for-128-languages) by *Arun Babu, Changhan Wang, Andros Tjandra, et al.* XLS-R used almost **half a million** hours of audio data in 128 languages for self-supervised pre-training and comes in sizes ranging from 300 milion up to **two billion** parameters. You can find the pretrained checkpoints on the 🤗 Hub:

- [**Wav2Vec2-XLS-R-300M**](https://huggingface.co/facebook/wav2vec2-xls-r-300m)
- [**Wav2Vec2-XLS-R-1B**](https://huggingface.co/facebook/wav2vec2-xls-r-1b)
- [**Wav2Vec2-XLS-R-2B**](https://huggingface.co/facebook/wav2vec2-xls-r-2b)

Similar to [BERT's masked language modeling objective](http://jalammar.github.io/illustrated-bert/), XLS-R learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network during self-supervised pre-training (*i.e.* diagram on the left below).

For fine-tuning, a single linear layer is added on top of the pre-trained network to train the model on labeled data of audio downstream tasks such as speech recognition, speech translation and audio classification (*i.e.* diagram on the right below).

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xls_r.png)

XLS-R shows impressive improvements over previous state-of-the-art results on both speech recognition, speech translation and speaker/language identification, *cf.* with Table 3-6, Table 7-10, and Table 11-12 respectively of the official [paper](https://ai.facebook.com/blog/xls-r-self-supervised-speech-processing-for-128-languages).

## Notebook Setup


In this notebook, we will give an in-detail explanation of how XLS-R - more specifically the pre-trained checkpoint [**Wav2Vec2-XLS-R-300M**](https://huggingface.co/facebook/wav2vec2-xls-r-300m) - can be fine-tuned for ASR.  

For demonstration purposes, we fine-tune the model on the low resource ASR dataset of [Common Voice](https://huggingface.co/datasets/common_voice) that contains only *ca.* 4h of validated training data.

XLS-R is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems, such as ASR and handwriting recognition.

I highly recommend reading the well-written blog post [*Sequence Modeling with CTC (2017)*](https://distill.pub/2017/ctc/) by Awni Hannun.

First, let's try to get a good GPU in our colab! With Google Colab's free version it's sadly becoming much harder to get access to a good GPU. With Google Colab Pro, however, one should easily get either a V100 or P100 GPU.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan 18 18:00:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0 Off |                  Off |
|  0%   28C    P8              38W / 480W |     14MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Before we start, let's install `datasets` and `transformers`. Also, we need the `torchaudio` to load audio files and `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

We strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training.

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Then you need to install Git-LFS to upload your model checkpoints:

In [ ]:
%%capture
!apt install git-lfs




---

${}^1$ In the [paper](https://arxiv.org/pdf/2006.13979.pdf), the model was evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text.

In 🤗 Transformers, the XLS-R model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer to decode the predicted output classes to the output transcription.

### Create `Wav2Vec2CTCTokenizer`

A pre-trained XLS-R model maps the speech signal to a sequence of context representations as illustrated in the figure above. However, for speech recognition the model has to to map this sequence of context representations to its corresponding transcription which means that a linear layer has to be added on top of the transformer block (shown in yellow in the diagram above). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *'BERT'* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend onXLS-R's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at the chosen dataset of Common Voice and define a vocabulary based on the transcriptions.

First, let's go to Common Voice [official website](https://commonvoice.mozilla.org/en/datasets) and pick a language to fine-tune XLS-R on. For this notebook, we will use Turkish.

For each language-specific dataset, you can find a language code corresponding to your chosen language. On [Common Voice](https://commonvoice.mozilla.org/en/datasets), look for the field "Version". The language code then corresponds to the prefix before the underscore. For Turkish, *e.g.* the language code is `"tr"`.

Great, now we can use 🤗 Datasets' simple API to download the data. The dataset name is `"common_voice"`, the configuration name corresponds to the language code, which is `"tr"` in our case.

Common Voice has many different splits including `invalidated`, which refers to data that was not rated as "clean enough" to be considered useful. In this notebook, we will only make use of the splits `"train"`, `"validation"` and `"test"`.

Because the Turkish dataset is so small, we will merge both the validation and training data into a training dataset and only use the test data for validation.

In [2]:
import os
hfhome_dir=os.path.join(os.sep, 'DATA10T', 'Cache')
#os.environ['TRANSFORMERS_CACHE'] = hfhome_dir
os.environ['HF_HOME'] = hfhome_dir
#os.environ['HF_HUB_CACHE'] = os.path.join(hfhome_dir, 'hub')
os.environ['HF_DATASETS_CACHE'] = hfhome_dir
#HF_HUB_OFFLINE=1
mycache_dir = hfhome_dir

In [3]:
from datasets import load_dataset, load_metric, Audio
dataset_name = "mozilla-foundation/common_voice_11_0"
Test_SAMPLES = 200
Train_SAMPLES=10000
train_dataset = load_dataset(dataset_name, "en", split=f"train[:{Train_SAMPLES}]", cache_dir=mycache_dir)
test_dataset = load_dataset(dataset_name, "en", split=f"test[:{Test_SAMPLES}]", cache_dir=mycache_dir)

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:1429: Futur

In [4]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "en"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-english" #https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-english
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID, cache_dir=mycache_dir)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID, cache_dir=mycache_dir)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
Y

In [5]:
test_dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 200
})

In [6]:
test_dataset['audio']

[{'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_27710027.mp3',
  'array': array([-7.95807864e-13, -1.59161573e-12, -6.25277607e-12, ...,
          1.52416442e-06,  1.94649760e-06,  1.25737506e-06]),
  'sampling_rate': 48000},
 {'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_699711.mp3',
  'array': array([ 0.00000000e+00, -7.05920511e-14, -5.47149741e-14, ...,
         -8.53668247e-09,  2.06165751e-09,  1.62003939e-08]),
  'sampling_rate': 48000},
 {'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_21953345.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          4.44706211e-06,  1.88029878e-06, -5.69346810e-08]),
  'sampling_rate': 48000},
 {'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423

In [7]:
test_dataset['sentence']

['Joe Keaton disapproved of films, and Buster also had reservations about the medium.',
 "She'll be all right.",
 'six',
 "All's well that ends well.",
 'It is a busy market town that serves a large surrounding area.',
 'The team had Olympic Champion Carolina Marin in their squad for the season.',
 'Do you mean it?',
 'The new patch is less invasive than the old one, but still causes regressions.',
 'How is Mozilla going to handle ambiguities like queue and cue?',
 "I guess you must think I'm kinda batty.",
 'No one near the remote machine you could ring?',
 'Sauce for the goose is sauce for the gander.',
 'Groves started writing songs when she was four years old.',
 'His uncle Mike Shula was the former head coach at Alabama.',
 'The response was that the Board did not appoint individual officers.',
 'The long-lived bridge still stands today.',
 'There was nothing to hold him back except himself.',
 'The meat processing and construction industries would be allowed to operate with reduc

In [8]:
sample=test_dataset[1]
sample

{'client_id': '0013037a1d45cc33460806cc3f8ecee9d536c45639ba4cbbf1564f1c051f53ff3c9f89ef2f1bf04badf55b3a2e7654c086f903681a7b6299616cff6f67598eff',
 'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_699711.mp3',
 'audio': {'path': '/DATA10T/Cache/downloads/extracted/ba5c265b423696aeece652c80df565d72d9f7c1cc9b807e127f44f989150f409/en_test_0/common_voice_en_699711.mp3',
  'array': array([ 0.00000000e+00, -7.05920511e-14, -5.47149741e-14, ...,
         -8.53668247e-09,  2.06165751e-09,  1.62003939e-08]),
  'sampling_rate': 48000},
 'sentence': "She'll be all right.",
 'up_votes': 2,
 'down_votes': 1,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'en',
 'segment': ''}

In [9]:
sample["audio"]["sampling_rate"]

48000

In [10]:
sample['sentence']

"She'll be all right."

In [11]:
inputids=processor.tokenizer(sample['sentence'])

In [35]:
inputids

{'input_ids': [3, 14, 11, 5, 18, 18, 4, 8, 11, 4, 7, 18, 18, 4, 24, 15, 13, 14, 26, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
decoded_str = processor.decode(inputids.input_ids)
decoded_str

2024-01-18 16:27:18.238713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 16:27:18.325525: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-18 16:27:18.607383: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-18 16:27:18.607425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

"<unk>he'l be al right<unk>"

S and . is replaced as "<unk>"

In [13]:
inputids=processor.tokenizer(sample['sentence'].lower())
decoded_str = processor.decode(inputids.input_ids)
decoded_str

"she'l be al right<unk>"

In [14]:
decoded_str = processor.tokenizer.decode(inputids.input_ids, group_tokens=False)
decoded_str

"she'll be all right<unk>"

In [16]:
decoded_str = processor.tokenizer.batch_decode([inputids.input_ids], group_tokens=False)
decoded_str

["she'll be all right<unk>"]

In [15]:
processor.tokenizer.encoder

{'<pad>': 0,
 '<s>': 1,
 '</s>': 2,
 '<unk>': 3,
 '|': 4,
 "'": 5,
 '-': 6,
 'a': 7,
 'b': 8,
 'c': 9,
 'd': 10,
 'e': 11,
 'f': 12,
 'g': 13,
 'h': 14,
 'i': 15,
 'j': 16,
 'k': 17,
 'l': 18,
 'm': 19,
 'n': 20,
 'o': 21,
 'p': 22,
 'q': 23,
 'r': 24,
 's': 25,
 't': 26,
 'u': 27,
 'v': 28,
 'w': 29,
 'x': 30,
 'y': 31,
 'z': 32}

In [17]:
speech_array, sampling_rate = librosa.load(sample["path"], sr=16_000)
#sample["audio"]["array"] is 48000
inputs_sample = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt")

In [18]:
inputs_sample

{'input_values': tensor([[4.2237e-05, 4.2201e-05, 4.2325e-05,  ..., 4.2364e-05, 4.2385e-05,
         4.2037e-05]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}

In [19]:
with torch.no_grad():
    logits = model(inputs_sample.input_values, attention_mask=inputs_sample.attention_mask).logits

In [20]:
logits.shape

torch.Size([1, 108, 33])

In [21]:
predicted_ids = torch.argmax(logits, dim=-1)
predicted_ids.shape

torch.Size([1, 108])

In [22]:
predicted_sentences = processor.batch_decode(predicted_ids)
predicted_sentences

["she'll be all right"]

In [23]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["sentence"]#.upper()
    return batch

test_dataset_new = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset_new["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [25]:
predicted_sentences

['joe keatson disapproved of films and buster also had reservations about the medium',
 "she'll be all right",
 'six',
 'all as well that ends well',
 'it is a busy market town that serves a large surrounded area',
 'the team at olympic chapion carolina marine in is quiet forty season',
 'do you mean it',
 'the new patch is less invasive than the old one but still causes regression',
 'how is moslillar going to handle andbewooth his like q and q',
 'christian ws sitin pam on te batty',
 'no one near the remote machine you could ring',
 'sauce for the guice is sauce for the gonder',
 'grafs started writing songs when she was four years old',
 'his uncle mike schula was the former edge coach at alabama',
 'the response was that the ball did not appoint individual officers',
 'the long-lived bleached still stands today',
 'there was nothing to hold him back except himself',
 'the mead processing and construction industries would be allowed to operate with readios personel',
 "cole's branc

In [26]:
for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Reference:", test_dataset[i]["sentence"])
    print("Prediction:", predicted_sentence)

----------------------------------------------------------------------------------------------------
Reference: Joe Keaton disapproved of films, and Buster also had reservations about the medium.
Prediction: joe keatson disapproved of films and buster also had reservations about the medium
----------------------------------------------------------------------------------------------------
Reference: She'll be all right.
Prediction: she'll be all right
----------------------------------------------------------------------------------------------------
Reference: six
Prediction: six
----------------------------------------------------------------------------------------------------
Reference: All's well that ends well.
Prediction: all as well that ends well
----------------------------------------------------------------------------------------------------
Reference: It is a busy market town that serves a large surrounding area.
Prediction: it is a busy market town that serves a large su

Many ASR datasets only provide the target text, `'sentence'` for each audio array `'audio'` and file `'path'`. Common Voice actually provides much more information about each audio file, such as the `'accent'`, etc. Keeping the notebook as general as possible, we only consider the transcribed text for fine-tuning.



In [4]:
train_dataset = train_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
test_dataset = test_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(train_dataset.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,"By this time they were found on many large buildings, including private houses."
1,Witteles appeared on the front page of The Washington Post about the same matter.
2,Europe has always fascinated and attracted.
3,The capital is Salima.
4,"""Iron Duke"" was also open to the public."
5,The player won a prize for correctly identifying at least three of the games.
6,It was named after the ancient village of Bethany.
7,I’m sure I shan’t be able!
8,"Fetal viability depends largely on the fetal organ maturity, and environmental conditions."
9,General Orlando B. Willcox's brigade when the latter was wounded in battle.


Alright! The transcriptions look fairly clean. Having translated the transcribed sentences, it seems that the language corresponds more to written-out text than noisy dialogue. This makes sense considering that [Common Voice](https://huggingface.co/datasets/common_voice) is a crowd-sourced read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

Let's simply remove all characters that don't contribute to the meaning of a word and cannot really be represented by an acoustic sound and normalize the text.

In [7]:
import re
#chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\”\�\']'


def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [8]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

Let's look at the processed text labels again.

In [9]:
show_random_elements(train_dataset.remove_columns(["path","audio"]))

,sentence
0,in the finals they were eliminated by the red wings four games to none
1,henleys grandfather sir robert henley was master of the court of the kings bench
2,witteles appeared on the front page of the washington post about the same matter
3,the piano nobile is accessed by an imposing stairwell
4,in high school he played baseball volleyball track and football
5,our best bet is that this animal is only active at night
6,the latter observational method is more suitable for global mapping
7,it is also a popular recreation destination for watercraft fishing hiking and climbing
8,twelve fighters and two torpedo bombers could be carried
9,bering died on the island along with much of the crew


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

Before finalizing the pre-processing, it is always advantageous to consult a native speaker of the target language to see whether the text can be further simplified.
For this blog post, [Merve](https://twitter.com/mervenoyann) was kind enough to take a quick look and noted that "hatted" characters - like `â` - aren't really used anymore in Turkish and can be replaced by their "un-hatted" equivalent, *e.g.* `a`.

This means that we should replace a sentence like `"yargı sistemi hâlâ sağlıksız"` to `"yargı sistemi hala sağlıksız"`.

Let's write another short mapping function to further simplify the text labels. Remember - the simler the text labels, the easier it is for the model to learn to predict those labels.



In [10]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

In [11]:
train_dataset = train_dataset.map(replace_hatted_characters)
test_dataset = test_dataset.map(replace_hatted_characters)

In [12]:
train_dataset[0]

{'path': '/DATA10T/Cache/downloads/extracted/b346aab12d7b8d3067d379267bbecbd285cb56b9272116d4588c4bf6e461e436/en_train_0/common_voice_en_19121553.mp3',
 'audio': {'path': '/DATA10T/Cache/downloads/extracted/b346aab12d7b8d3067d379267bbecbd285cb56b9272116d4588c4bf6e461e436/en_train_0/common_voice_en_19121553.mp3',
  'array': array([ 0.00000000e+00,  1.96611932e-15, -7.55335397e-14, ...,
          1.68718216e-05,  1.41903001e-07,  1.16676725e-04]),
  'sampling_rate': 48000},
 'sentence': 'the track appears on the compilation album kraftworks'}

In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [14]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [15]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [16]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '(': 1,
 ')': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 'á': 29,
 'è': 30,
 'é': 31,
 'í': 32,
 'ï': 33,
 'ó': 34,
 'ō': 35,
 'ř': 36,
 'ž': 37,
 '–': 38,
 '—': 39,
 '‘': 40,
 '’': 41}

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `""` and `'`. Note that we did not exclude those special characters because:

The model has to learn to predict when a word is finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.

One should always keep in mind that pre-processing is a very important step before training your model. E.g., we don't want our model to differentiate between `a` and `A` just because we forgot to normalize the data. The difference between `a` and `A` does not depend on the "sound" of the letter at all, but more on grammatical rules - *e.g.* use a capitalized letter at the beginning of the sentence. So it is sensible to remove the difference between capitalized and non-capitalized letters so that the model has an easier time learning to transcribe speech.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Common Voice's training set.

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [18]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

44

Cool, now our vocabulary is complete and consists of 39 tokens, which means that the linear layer that we will add on top of the pretrained XLS-R checkpoint will have an output dimension of 39.

Let's now save the vocabulary as a json file.

In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to load the vocabulary into an instance of the `Wav2Vec2CTCTokenizer` class.

In [20]:
from transformers import Wav2Vec2CTCTokenizer, AutoTokenizer, AutoConfig

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
model_checkpoint="facebook/wav2vec2-xls-r-300m"
# config = AutoConfig.from_pretrained(model_checkpoint)
# tokenizer = AutoTokenizer.from_pretrained(
#                 "./", #vocab_filepath,
#                 #**tokenizer_kwargs,
#                 cache_dir = mycache_dir,
#                 config=config,#None
#                 # tokenizer_type=tokenizer_type, #'wav2vec2'
#                 #do_lower_case=True,
#                 unk_token="[UNK]",
#                 pad_token="[PAD]",
#                 word_delimiter_token="|",
#                 )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
tokenizer

Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=44, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]}, clean_up_tokenization_spaces=True)

In [22]:
tokenizer.encoder

{'(': 1,
 ')': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 'á': 29,
 'è': 30,
 'é': 31,
 'í': 32,
 'ï': 33,
 'ó': 34,
 'ō': 35,
 'ř': 36,
 'ž': 37,
 '–': 38,
 '—': 39,
 '‘': 40,
 '’': 41,
 '|': 0,
 '[UNK]': 42,
 '[PAD]': 43}

In [23]:
len(tokenizer.encoder)

44

In [24]:
sample = train_dataset[0]

In [25]:
inputids=tokenizer(sample['sentence'].lower())
decoded_str = tokenizer.decode(inputids.input_ids)
decoded_str

2024-01-18 18:02:14.957700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 18:02:15.032232: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2024-01-18 18:02:15.476228: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-18 18:02:15.476357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-01-18 18:02:15.476359: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


'the track apears on the compilation album kraftworks'

In [26]:
decoded_str = tokenizer.decode(inputids.input_ids, group_tokens=False)
decoded_str

'the track appears on the compilation album kraftworks'

In [27]:
tokenizer.save_pretrained("./output")

('./output/tokenizer_config.json',
 './output/special_tokens_map.json',
 './output/vocab.json',
 './output/added_tokens.json')

If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the [🤗 Hub](https://huggingface.co/). Let's call the repo to which we will upload the files
`"wav2vec2-large-xlsr-turkish-demo-colab"`:

In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-turkish-colab"

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/).

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:718: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:718: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/patrickvonplaten/wav2vec2-large-xls-r-300m-turkish-colab into local empty directory.
To https://huggingface.co/patrickvonplaten/wav2vec2-large-xls-r-300m-turkish-colab
   54dbd42..56ebe74  main -> main



'https://huggingface.co/patrickvonplaten/wav2vec2-large-xls-r-300m-turkish-colab/commit/56ebe744b5acf5853233e4425bd914b938c4f013'

Great, you can see the just created repository under `https://huggingface.co/<your-username>/wav2vec2-large-xls-r-300m-tr-colab`

### Create `Wav2Vec2FeatureExtractor`

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus,
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLS-R was pretrained on audio data of [Babel](http://www.reading.ac.uk/AcaDepts/ll/speechlab/babel/r),
[Multilingual LibriSpeech (MLS)](https://huggingface.co/datasets/multilingual_librispeech), [Common Voice](https://huggingface.co/datasets/common_voice), [VoxPopuli](https://arxiv.org/abs/2101.00390), and [VoxLingua107](https://arxiv.org/abs/2011.12998) at a sampling rate of 16kHz. Common Voice, in its original form, has a sampling rate of 48kHz, thus we will have to downsample the fine-tuning data to 16kHz in the following.



A `Wav2Vec2FeatureExtractor` object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLS-R models checkpoints should **always** use the `attention_mask`.

In [28]:
from transformers import Wav2Vec2FeatureExtractor, AutoFeatureExtractor

# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

model_checkpoint="facebook/wav2vec2-xls-r-300m"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint, cache_dir=mycache_dir, do_normalize=True,return_attention_mask=True)

In [29]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

Great, XLS-R's feature extraction pipeline is thereby fully defined!

For improved user-friendliness, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [30]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to `sentence`, our datasets include two more column names `path` and `audio`. `path` states the absolute path of the audio file. Let's take a look.


In [31]:
train_dataset[0]["path"]

'/DATA10T/Cache/downloads/extracted/b346aab12d7b8d3067d379267bbecbd285cb56b9272116d4588c4bf6e461e436/en_train_0/common_voice_en_19121553.mp3'

XLS-R expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically by calling the other column `audio`. Let try it out.

In [32]:
train_dataset[0]["audio"]

{'path': '/DATA10T/Cache/downloads/extracted/b346aab12d7b8d3067d379267bbecbd285cb56b9272116d4588c4bf6e461e436/en_train_0/common_voice_en_19121553.mp3',
 'array': array([ 0.00000000e+00,  1.96611932e-15, -7.55335397e-14, ...,
         1.68718216e-05,  1.41903001e-07,  1.16676725e-04]),
 'sampling_rate': 48000}

In [58]:
train_dataset

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 10000
})

Great, we can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

In the example above we can see that the audio data is loaded with a sampling rate of 48kHz whereas 16kHz are expected by the model. We can set the audio feature to the correct sampling rate by making use of [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column):

In [33]:
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16_000))

Let's take a look at `"audio"` again.

In [34]:
train_dataset[0]["audio"]

{'path': '/DATA10T/Cache/downloads/extracted/b346aab12d7b8d3067d379267bbecbd285cb56b9272116d4588c4bf6e461e436/en_train_0/common_voice_en_19121553.mp3',
 'array': array([ 7.27595761e-11, -2.32830644e-10, -4.65661287e-10, ...,
        -5.81125205e-05, -2.75571801e-05,  2.96054786e-05]),
 'sampling_rate': 16000}

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [35]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset)-1)

print(train_dataset[rand_int]["sentence"])
ipd.Audio(data=train_dataset[rand_int]["audio"]["array"], autoplay=True, rate=16000)

motilal later joined the swaraj party which sought to enter the britishsponsored councils


It seems like the data is now correctly loaded and resampled.

It can be heard, that the speakers change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [36]:
rand_int = random.randint(0, len(train_dataset)-1)

print("Target text:", train_dataset[rand_int]["sentence"])
print("Input array shape:", train_dataset[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_dataset[rand_int]["audio"]["sampling_rate"])

Target text: the companies supplying this meat were known as the big four of meatpacking
Input array shape: (100224,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can leverage `Wav2Vec2Processor` to process the data to the format expected by `Wav2Vec2ForCTC` for training. To do so let's make use of Dataset's [`map(...)`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=map#datasets.DatasetDict.map) function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [37]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    # with processor.as_target_processor():
    #     batch["labels"] = processor(batch["sentence"]).input_ids
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [38]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset =test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

Long input sequences require a lot of memory. XLS-R is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). In case this demo crashes with an "Out-of-memory" error for you, you might want to uncomment the following lines to filter all sequences that are longer than 5 seconds for training.

In [ ]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Awesome, now we are ready to start training!

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLS-R has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLS-R requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/7e61d56a45c19284cfda0cee8995fb552f6b1f4e/examples/pytorch/speech-recognition/run_speech_recognition_ctc.py#L219).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLS-R processor's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [39]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [40]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [41]:
wer_metric = load_metric("wer")

/tmp/ipykernel_2893126/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [42]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained checkpoint of [Wav2Vec2-XLS-R-300M](https://huggingface.co/facebook/wav2vec2-xls-r-300m). The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small (~6h of training data) and because Common Voice is quite noisy, fine-tuning Facebook's [wav2vec2-xls-r-300m checkpoint](https://huggingface.co/facebook/wav2vec2-xls-r-300m) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough.

**Note**: When using this notebook to train XLS-R on another language of Common Voice those hyper-parameter settings might not work very well. Feel free to adapt those depending on your use case.

In [81]:
del model

In [43]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    cache_dir = mycache_dir,
    # attention_dropout=0.0,
    # hidden_dropout=0.0,
    # feat_proj_dropout=0.0,
    # mask_time_prob=0.05,
    # layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLS-R consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [44]:
model.freeze_feature_extractor()

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [51]:
#model.freeze_feature_encoder()#model.freeze_feature_extractor()

In [52]:
#model.freeze_base_model()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/DATA10T/output/wave2vec2xls2",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

In [45]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="/DATA10T/output/wave2vec2xls3",
    group_by_length=True,#makes training more efficient by grouping training samples of similar input length into one batch.
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    warmup_ratio=0.1,
    #warmup_steps=500,
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    #metric_for_best_model=myEvaluator.#"accuracy",
    fp16=True,
    push_to_hub=False,
    #gradient_checkpointing=True,#reduce GPU memory, or use model.gradient_checkpointing_enable()
)

In [44]:
#model.gradient_checkpointing_enable()

Now, all instances can be passed to Trainer and we are ready to start training!

In [46]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take multiple hours depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is just to demonstrate how to fine-tune XLS-R on an ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 8 or even less and increase [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [47]:
trainer.train()

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Wer
1,2.911300,2.549300,1.000000
2,1.085700,1.121199,0.703566
3,0.803900,0.912603,0.584353
4,0.641600,0.900724,0.529005
5,0.533900,0.813429,0.483768
6,0.459200,0.818006,0.477382
7,0.385800,0.759845,0.439063
8,0.340000,0.778941,0.443853
9,0.301200,0.796572,0.428952
10,0.255400,0.776158,0.414050


/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transform

TrainOutput(global_step=6250, training_loss=0.9754180757141113, metrics={'train_runtime': 2831.0604, 'train_samples_per_second': 35.322, 'train_steps_per_second': 2.208, 'total_flos': 1.905400461749447e+19, 'train_loss': 0.9754180757141113, 'epoch': 10.0})

In [49]:
trainer.train()#new

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
400,5.032400,3.010644,1.000000
800,1.965400,1.089912,0.675359
1200,0.869600,0.906917,0.565194
1600,0.640900,0.851737,0.496541
2000,0.517900,0.840233,0.459819
2400,0.447900,0.789620,0.464077
2800,0.390200,0.845395,0.462480
3200,0.347700,0.843843,0.431080
3600,0.303800,0.843381,0.428419
4000,0.282100,0.890193,0.437467


/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155

: 

In [48]:
trainer.evaluate()

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 0.7598454356193542,
 'eval_wer': 0.4390633315593401,
 'eval_runtime': 4.544,
 'eval_samples_per_second': 44.014,
 'eval_steps_per_second': 5.502,
 'epoch': 10.0}

The training loss and validation WER go down nicely.

You can now upload the result of the training to the 🤗 Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-turkish-colab
Configuration saved in wav2vec2-large-xls-r-300m-turkish-colab/config.json
Model weights saved in wav2vec2-large-xls-r-300m-turkish-colab/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-turkish-colab/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.35k/1.18G [00:00<?, ?B/s]

Upload file runs/Nov12_14-33-48_c2d4142f9305/events.out.tfevents.1636727730.c2d4142f9305.1468.0:  41%|####1   …

To https://huggingface.co/patrickvonplaten/wav2vec2-large-xls-r-300m-turkish-colab
   56ebe74..fe76946  main -> main

Dropping the following result as it does not have all the necessary field:
{'dataset': {'name': 'common_voice', 'type': 'common_voice', 'args': 'tr'}}
To https://huggingface.co/patrickvonplaten/wav2vec2-large-xls-r-300m-turkish-colab
   fe76946..5f0d67b  main -> main



'https://huggingface.co/patrickvonplaten/wav2vec2-large-xls-r-300m-turkish-colab/commit/fe769461e4e2fb9534740e6c278a0cfabf268474'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
```

For more examples of how XLS-R can be fine-tuned, please take a look at the [official speech recognition examples](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#examples).

### Evaluation

As a final check, let's load the model and verify that it indeed has learned to transcribe Turkish speech.

Let's first load the pretrained checkpoint.

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

loading configuration file wav2vec2-large-xls-r-300m-turkish-colab/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "grad

In [99]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [100]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=31, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]}, clean_up_tokenization_spaces=True)

In [101]:
onesample=test_dataset[0]
onesample

{'input_values': [1.9871007680194452e-05,
  1.9876821170328185e-05,
  1.987294490390923e-05,
  1.9871007680194452e-05,
  1.98671332327649e-05,
  1.9851631805067882e-05,
  1.9876821170328185e-05,
  1.987488394661341e-05,
  1.9865196009050123e-05,
  1.984678783628624e-05,
  1.9871007680194452e-05,
  1.9868100935127586e-05,
  1.987294490390923e-05,
  1.987294490390923e-05,
  1.9863256966345944e-05,
  1.9876821170328185e-05,
  1.98923207790358e-05,
  1.985938251891639e-05,
  1.987294490390923e-05,
  1.9876821170328185e-05,
  1.9900071492884308e-05,
  1.9869070456479676e-05,
  1.9888446331606247e-05,
  1.9898134269169532e-05,
  1.9894258002750576e-05,
  1.9882632841472514e-05,
  1.988650910789147e-05,
  1.989329030038789e-05,
  1.9906852685380727e-05,
  1.9891836927854456e-05,
  1.9888446331606247e-05,
  1.9891353076673113e-05,
  1.9891353076673113e-05,
  1.9897164747817442e-05,
  1.9908789909095503e-05,
  1.9907822206732817e-05,
  1.9899101971532218e-05,
  1.9884571884176694e-05,
  1.98913

In [117]:
onesample['labels']

[10,
 15,
 5,
 0,
 11,
 5,
 1,
 20,
 15,
 14,
 0,
 4,
 9,
 19,
 1,
 16,
 16,
 18,
 15,
 22,
 5,
 4,
 0,
 15,
 6,
 0,
 6,
 9,
 12,
 13,
 19,
 0,
 1,
 14,
 4,
 0,
 2,
 21,
 19,
 20,
 5,
 18,
 0,
 1,
 12,
 19,
 15,
 0,
 8,
 1,
 4,
 0,
 18,
 5,
 19,
 5,
 18,
 22,
 1,
 20,
 9,
 15,
 14,
 19,
 0,
 1,
 2,
 15,
 21,
 20,
 0,
 20,
 8,
 5,
 0,
 13,
 5,
 4,
 9,
 21,
 13]

In [103]:
len(onesample["input_values"]) #list

100800

In [104]:
input_dict = processor(onesample["input_values"], return_tensors="pt", padding=True)

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [106]:
input_dict.input_values.shape

torch.Size([1, 100800])

In [107]:
model=model.cuda()


Now, we will just take the first example of the test set, run it through the model and take the `argmax(...)` of the logits to retrieve the predicted token ids.

In [108]:
logits = model(input_dict.input_values.to("cuda")).logits

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [109]:
logits.shape

torch.Size([1, 314, 33])

In [110]:
pred_ids = torch.argmax(logits, dim=-1)
pred_ids.shape

torch.Size([1, 314])

In [111]:
pred_id=pred_ids[0]

In [112]:
pred_id.shape

torch.Size([314])

In [116]:
predicted_sentence = processor.batch_decode(pred_id)
predicted_sentence

['[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'j',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'o',
 'o',
 'o',
 'l',
 'l',
 'l',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'c',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'a',
 'a',
 '[PAD]',
 '[PAD]',
 't',
 'e',
 's',
 's',
 '',
 '[PAD]',
 '[PAD]',
 'o',
 'o',
 'n',
 '[PAD]',
 '',
 '[PAD]',
 'd',
 '[PAD]',
 'i',
 's',
 's',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'a',
 'p',
 '[PAD]',
 '[PAD]',
 'p',
 'p',
 '[PAD]',
 'r',
 'r',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'o',
 'o',
 'o',
 '[PAD]',
 '[PAD]',
 'v',
 'e',
 'e',
 'd',
 '',
 '',
 '[PAD]',
 '[PAD]',
 'o',
 'o',
 'f',
 '',
 '',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 'f',
 '

In [115]:
predicted_sentence = processor.batch_decode(pred_ids)
predicted_sentence

['jolcates on disapproved of filrms and buster also had reervation ouf the medin']

In [121]:
len(onesample['labels'])

81

In [126]:
label_str=processor.batch_decode([onesample['labels']],group_tokens=False)
label_str

['joe keaton disapproved of films and buster also had reservations about the medium']

In [123]:
label_str=processor.batch_decode(onesample['labels'],group_tokens=False)
label_str

['j',
 'o',
 'e',
 '',
 'k',
 'e',
 'a',
 't',
 'o',
 'n',
 '',
 'd',
 'i',
 's',
 'a',
 'p',
 'p',
 'r',
 'o',
 'v',
 'e',
 'd',
 '',
 'o',
 'f',
 '',
 'f',
 'i',
 'l',
 'm',
 's',
 '',
 'a',
 'n',
 'd',
 '',
 'b',
 'u',
 's',
 't',
 'e',
 'r',
 '',
 'a',
 'l',
 's',
 'o',
 '',
 'h',
 'a',
 'd',
 '',
 'r',
 'e',
 's',
 'e',
 'r',
 'v',
 'a',
 't',
 'i',
 'o',
 'n',
 's',
 '',
 'a',
 'b',
 'o',
 'u',
 't',
 '',
 't',
 'h',
 'e',
 '',
 'm',
 'e',
 'd',
 'i',
 'u',
 'm']

In [124]:
label_str=processor.decode(onesample['labels'],group_tokens=False)
label_str

'joe keaton disapproved of films and buster also had reservations about the medium'

In [125]:
label_str=processor.tokenizer.decode(onesample['labels'],group_tokens=False)
label_str

'joe keaton disapproved of films and buster also had reservations about the medium'

We adapted `common_voice_test` quite a bit so that the dataset instance does not contain the original sentence label anymore. Thus, we re-use the original dataset to get the label of the first example.